### this script adjusts the true positive (currently funded papers) cases with addressing false negative cases (papers should have funding)

In [15]:
import pandas as pd
import itertools

In [6]:
ci_global_path = "../../data/NationalFunding/Data/RobustCheck/check_09_and_18/ci_global.csv"
ci_cntry_path = "../../data/NationalFunding/Data/RobustCheck/check_09_and_18/ci_us_chn.csv"
stats_path = "../../data/NationalFunding/Data/RobustCheck/check_09_and_18/global_us_chn_stats.csv"

In [7]:
ci_global_df = pd.read_csv(ci_path)
ci_df

,cntry,year,mean,lower,upper
0,global,2009,0.162040,0.16022,0.163860
1,global,2018,0.054213,0.05302,0.055407


In [8]:
ci_cntry_df = pd.read_csv(ci_cntry_path)
ci_cntry_df.head()

,cntry,year,mean,lower,upper
0,china,2009,0.299000,0.295908,0.302092
1,china,2018,0.204272,0.201793,0.206751
2,usa,2009,0.263453,0.261070,0.265835
3,usa,2018,0.106734,0.104741,0.108727


In [11]:
ci_df = pd.concat([ci_global_df,ci_cntry_df],axis=0)
ci_df.head(2)

,cntry,year,mean,lower,upper
0,global,2009,0.162040,0.16022,0.163860
1,global,2018,0.054213,0.05302,0.055407


In [13]:
stats_df = pd.read_csv(stats_path)
stats_df.head(2)

,cntry,year,type,value
0,global,2009,total,1038638
1,global,2009,funded,531320


In [29]:
def adjust_with_false_negative(total, funded, notfunded, mean, lower, upper):
    false_negative = notfunded*mean
    false_negative_lower = notfunded*lower
    false_negative_upper = notfunded*upper
    return funded+false_negative, funded+false_negative_lower, funded+false_negative_upper

In [16]:
stats_df = stats_df.pivot(index=['cntry','year'],columns='type',values='value').reset_index()
stats_df = stats_df.set_index(['cntry','year'])
ci_df = ci_df.set_index(['cntry','year'])

In [30]:
res = []
cntrys = stats_df.index.get_level_values(0).unique()
years = stats_df.index.get_level_values(1).unique()
for cntry, year in itertools.product(cntrys, years):
    total = stats_df.loc[(cntry,year),'total']
    funded = stats_df.loc[(cntry,year),'funded']
    notfunded = stats_df.loc[(cntry,year),'notfunded']
    mean = ci_df.loc[(cntry,year),'mean']
    lower = ci_df.loc[(cntry,year),'lower']
    upper = ci_df.loc[(cntry,year),'upper']
    estimated_funded, estimated_lower, estimated_upper = adjust_with_false_negative(total, funded, notfunded, mean, lower, upper)
    res.append([cntry,year,total,funded,estimated_funded,estimated_lower,estimated_upper])
res_df = pd.DataFrame(res, columns=['cntry','year','total','funded','funded_estimated','estimated_lower','estimated_upper'])

In [33]:
res_df.to_csv("../../data/NationalFunding/Data/RobustCheck/check_09_and_18/global_us_chn_stats_adjusted.csv", index=False)